In [1]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df

from sklearn.model_selection import train_test_split
from copy import deepcopy


# from project_modules.utils import get_logger

# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

In [2]:
from pathlib import Path

# input_path = Path("../lc-project-data")
# output_path = Path("../lc-project-data")

# for MP,
input_path = Path("../Data/DataV3")
output_path = Path("../Data/DataV3")

In [3]:
site_name_dict = {1: "LDN", 2: "MTL", 3: "SAN", 4: "RIO", 5: "LUS", 6: "CA", 7: "NA"}
status_name_dict = {
    0:'HC',
    1:'LC_NEG',
    2:'LC_POS'
}

In [4]:
# df81 = load_dataset_to_df('../Data/DataV2/featuresThresAll81.arrow', verbose = True)
# df85 = load_dataset_to_df('../Data/DataV2/featuresThresAll85.arrow', verbose = True)
# dfDT = load_dataset_to_df("../Data/DataV2/featuresDT.arrow", verbose=True)

df = load_dataset_to_df(input_path/"2024-04-30-features-cmplt.arrow", verbose=True)

In [5]:
import json

with open('JSON/dict-preLongCovid.json', 'r') as file:
    lColPreLC = json.load(file)

In [6]:
lColPreLC2 = deepcopy(lColPreLC)

In [7]:
lColPreLC2.remove('CD-mode_of_diag')

### Do train-test-holdout split here and save as a new dataset

In [8]:
print('% of patiens from each site in cmplt')
pd.crosstab(df["LC_STATUS"].map(status_name_dict), df["SITE"].map(site_name_dict), normalize="columns").round(2)

% of patiens from each site in cmplt


SITE,LDN,LUS,MTL,RIO,SAN
LC_STATUS,,,,,
HC,0.55,0.75,0.69,0.16,0.21
LC_NEG,0.35,0.22,0.31,0.83,0.78
LC_POS,0.10,0.03,0.00,0.01,0.01


In [9]:
print("# of patiens from each site in cmplt")

pd.crosstab(df["LC_STATUS"].map(status_name_dict), df["SITE"].map(site_name_dict))

# of patiens from each site in cmplt


SITE,LDN,LUS,MTL,RIO,SAN
LC_STATUS,,,,,
HC,176,934,1997,128,71
LC_NEG,110,268,916,650,266
LC_POS,32,38,0,9,3


In [10]:
def straify_train_test_holdout(df, labelVar, stratifyCol, test_size = 0.2, holdout_size = 0.2, random_state = 42 ):
    from sklearn.model_selection import train_test_split
    df_train, df_holdout = train_test_split(df, test_size = holdout_size, random_state = random_state, stratify = df['LC_STATUS'])
    # Split the data into training and holdout sets
    df_train, df_holdout = train_test_split(
        df,
        test_size=holdout_size,
        stratify=df[stratifyCol],
        random_state=42,
    )

    df_train, df_test = train_test_split(
        df_train,
        test_size=test_size / (1 - holdout_size),
        random_state=random_state,
        stratify=df_train[stratifyCol],
    )
    return df_train, df_test, df_holdout

In [11]:
# Create combined variable to stratify on both LC_STATUS and SITE
df["LC_STATUS_SITE"] = df["LC_STATUS"].astype(str) + "_" + df["SITE"].astype(str)

df_train, df_test, df_holdout = straify_train_test_holdout(
    df,
    "LC_STATUS",
    "LC_STATUS_SITE",
    test_size=0.2,
    holdout_size=0.2,
    random_state=42,
)

Verfiy stratification accurate

In [12]:
pd.crosstab(df_train["LC_STATUS"], df_train["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
0,0.552083,0.685355,0.209756,0.161359,0.754717
1,0.343750,0.314645,0.780488,0.828025,0.215633
2,0.104167,0.000000,0.009756,0.010616,0.029650


In [13]:
pd.crosstab(df_test["LC_STATUS"], df_test["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
0,0.555556,0.686106,0.208955,0.164557,0.751004
1,0.349206,0.313894,0.791045,0.822785,0.216867
2,0.095238,0.000000,0.000000,0.012658,0.032129


In [14]:
pd.crosstab(df_holdout["LC_STATUS"], df_holdout["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
0,0.555556,0.685567,0.205882,0.164557,0.751004
1,0.349206,0.314433,0.779412,0.822785,0.216867
2,0.095238,0.000000,0.014706,0.012658,0.032129


In [15]:
df

,__index_level_0__,SITE,LC_STATUS,sex,age,MED-Analgesics,MED-Anesthetics,MED-Antibacterials,MED-Anticonvulsants,MED-Antidepressants,...,LABS-IND-sodium,LABS-IND-triglycerides,LABS-IND-urea,LABS-IND-wbc,SUBST-1,SUBST-2,SUBST-3,SUBST-4,SUBST-5,LC_STATUS_SITE
0,0,2,1,2.0,30.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1_2
1,1,2,1,1.0,33.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1_2
2,2,2,1,2.0,36.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1_2
3,3,2,1,2.0,50.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1_2
4,4,2,1,1.0,40.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5593,5659,3,1,1.0,48.0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1_3
5594,5660,3,1,2.0,42.0,0,0,0,0,0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1_3
5595,5661,3,1,1.0,23.0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1_3
5596,5662,3,1,1.0,71.0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1_3


In [16]:
datasets.arrow_dataset.Dataset.from_pandas(df_train,preserve_index=False).save_to_disk(
    output_path/"TTS/dfcmplt_Train.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(df_test, preserve_index=False).save_to_disk(
    output_path / "TTS/dfcmplt_Test.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(
    df_holdout, preserve_index=False
).save_to_disk(output_path / "TTS/dfcmplt_Holdout.arrow")

Saving the dataset (0/1 shards):   0%|          | 0/3358 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1120 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1120 [00:00<?, ? examples/s]

In [17]:
datasets.arrow_dataset.Dataset.from_pandas(df_train[lColPreLC],preserve_index=False).save_to_disk(
    output_path/"TTS/dfcmpltPreLC_Train.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(df_test[lColPreLC], preserve_index=False).save_to_disk(
    output_path / "TTS/dfcmpltPreLC_Test.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(
    df_holdout[lColPreLC], preserve_index=False
).save_to_disk(output_path / "TTS/dfcmpltPreLC_Holdout.arrow")

Saving the dataset (0/1 shards):   0%|          | 0/3358 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1120 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1120 [00:00<?, ? examples/s]

In [18]:
# lColPreLC2 = remove mode of diag from prelc cols
datasets.arrow_dataset.Dataset.from_pandas(df_train[lColPreLC2],preserve_index=False).save_to_disk(
    output_path/"TTS/dfcmpltPreLC2_Train.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(df_test[lColPreLC2], preserve_index=False).save_to_disk(
    output_path / "TTS/dfcmpltPreLC2_Test.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(
    df_holdout[lColPreLC2], preserve_index=False
).save_to_disk(output_path / "TTS/dfcmpltPreLC2_Holdout.arrow")

Saving the dataset (0/1 shards):   0%|          | 0/3358 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1120 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1120 [00:00<?, ? examples/s]

In [19]:
# lColPreLC3 = remove 21 patietnst that were incorectly included - May 2/3

# reimport original data
from pathlib import Path
input_path = Path("../Data/Original Data/2024_04_26")


dfOrg = pd.read_excel(input_path / "FINAL-CORRECTED-DATA-2024-04-26.xlsx")

#records to remove
recRemove = ['353','363','400','1020','1147','1154','1159','1164','1171','1173','1228','1242','1272','1300','1325','1344','1345','1346','1348','1349','1380']
recRemove = [int(x) for x in recRemove]
indexOrg = dfOrg[~dfOrg['record_id'].isin(recRemove)].index
recRemove = [int(x) for x in recRemove]
# dfOrg[['record_id','__index_']]



df_train_LC3 = df_train[df_train['__index_level_0__'].isin(indexOrg)]
df_test_LC3 = df_test[df_test['__index_level_0__'].isin(indexOrg)]
df_holdout_LC3 = df_holdout[df_holdout['__index_level_0__'].isin(indexOrg)]

lColPreLC3 = [x for x in lColPreLC2 if 'covid_test' not in  x.lower()]



datasets.arrow_dataset.Dataset.from_pandas(df_train_LC3[lColPreLC3],preserve_index=False).save_to_disk(
    output_path/"TTS/dfcmpltPreLC3_Train.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(df_test_LC3[lColPreLC3], preserve_index=False).save_to_disk(
    output_path / "TTS/dfcmpltPreLC3_Test.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(
    df_holdout_LC3[lColPreLC3], preserve_index=False
).save_to_disk(output_path / "TTS/dfcmpltPreLC3_Holdout.arrow")


Saving the dataset (0/1 shards):   0%|          | 0/3348 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1114 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1115 [00:00<?, ? examples/s]

In [20]:
for key in site_name_dict.keys():
    if key == 7:
        query = 'SITE==1 or SITE==2 or SITE==3'
    elif key == 6:
        query = "SITE==1 or SITE==2"
    else:
        query = f'SITE=={key}'

    datasets.arrow_dataset.Dataset.from_pandas(
        df_train.query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmplt_SITE{key}_Train.arrow")


    datasets.arrow_dataset.Dataset.from_pandas(
        df_test.query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmplt_SITE{key}_Test.arrow")

    datasets.arrow_dataset.Dataset.from_pandas(
        df_holdout.query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmplt_SITE{key}_Holdout.arrow")



    # Versions with no LongCovid cols
    datasets.arrow_dataset.Dataset.from_pandas(
        df_train[lColPreLC].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC_SITE{key}_Train.arrow")

    datasets.arrow_dataset.Dataset.from_pandas(
        df_test[lColPreLC].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC_SITE{key}_Test.arrow")

    datasets.arrow_dataset.Dataset.from_pandas(
        df_holdout[lColPreLC].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC_SITE{key}_Holdout.arrow")
    
    # lColPreLC2 = remove mode of diag from prelc cols
    datasets.arrow_dataset.Dataset.from_pandas(
        df_train[lColPreLC2].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC2_SITE{key}_Train.arrow")

    datasets.arrow_dataset.Dataset.from_pandas(
        df_test[lColPreLC2].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC2_SITE{key}_Test.arrow")

    datasets.arrow_dataset.Dataset.from_pandas(
        df_holdout[lColPreLC2].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC2_SITE{key}_Holdout.arrow")
    
    
    #lColPreLC3 = remove problematic patients
    datasets.arrow_dataset.Dataset.from_pandas(
        df_train_LC3[lColPreLC3].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC3_SITE{key}_Train.arrow")

    datasets.arrow_dataset.Dataset.from_pandas(
        df_test_LC3[lColPreLC3].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC3_SITE{key}_Test.arrow")

    datasets.arrow_dataset.Dataset.from_pandas(
        df_holdout_LC3[lColPreLC3].query(query), preserve_index=False
    ).save_to_disk(output_path / f"TTS/dfcmpltPreLC3_SITE{key}_Holdout.arrow")

Saving the dataset (0/1 shards):   0%|          | 0/192 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/63 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/63 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/192 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/63 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/713 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/713 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2135 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/707 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/708 [00:00<?, ? examples/s]

# ---- BELOW ARCHIVED April 30 
- based on changes made with filling in data with missing values and assusmptions
- Confirmed by emails to team on april 28 and 29


## Split T81

In [19]:
# Create combined variable to stratify on both LC_STATUS and SITE
df81["LC_STATUS_SITE"] = df81["LC_STATUS"] + "_" + df81["SITE"].astype(str)

df_train81, df_test81, df_holdout81 = straify_train_test_holdout(df81, 'LC_STATUS', 'LC_STATUS_SITE', test_size = 0.2, holdout_size = 0.2, random_state = 42)

Verfiy stratification accurate

In [20]:
pd.crosstab(df_train81["LC_STATUS"], df_train81["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.133803,0.000000,0.009756,0.0125,0.068966
LC_NEG,0.725352,0.685355,0.209756,0.0125,0.557994
LC_POS,0.140845,0.314645,0.780488,0.9750,0.373041


In [21]:
pd.crosstab(df_test81["LC_STATUS"], df_test81["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.145833,0.000000,0.000000,0.015038,0.074074
LC_NEG,0.729167,0.685567,0.208955,0.007519,0.555556
LC_POS,0.125000,0.314433,0.791045,0.977444,0.370370


In [22]:
pd.crosstab(df_holdout81["LC_STATUS"], df_holdout81["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.127660,0.000000,0.014706,0.015038,0.074074
LC_NEG,0.723404,0.686106,0.205882,0.007519,0.555556
LC_POS,0.148936,0.313894,0.779412,0.977444,0.370370


## Split T85

In [23]:
# Create combined variable to stratify on both LC_STATUS and SITE
df85["LC_STATUS_SITE"] = df85["LC_STATUS"] + "_" + df85["SITE"].astype(str)

df_train85, df_test85, df_holdout85 = straify_train_test_holdout(df85, 'LC_STATUS', 'LC_STATUS_SITE', test_size = 0.2, holdout_size = 0.2, random_state = 42)

Verfiy stratification accurate

In [24]:
pd.crosstab(df_train85["LC_STATUS"], df_train85["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.134752,0.000000,0.009756,0.0125,0.029610
LC_NEG,0.730496,0.685355,0.209756,0.0125,0.753701
LC_POS,0.134752,0.314645,0.780488,0.9750,0.216689


In [25]:
pd.crosstab(df_test85["LC_STATUS"], df_test85["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.142857,0.000000,0.000000,0.015038,0.032258
LC_NEG,0.714286,0.685567,0.208955,0.007519,0.754032
LC_POS,0.142857,0.314433,0.791045,0.977444,0.213710


In [26]:
pd.crosstab(df_holdout85["LC_STATUS"], df_holdout85["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.127660,0.000000,0.014706,0.015038,0.032129
LC_NEG,0.723404,0.686106,0.205882,0.007519,0.751004
LC_POS,0.148936,0.313894,0.779412,0.977444,0.216867


## Split DT

In [29]:
# Create combined variable to stratify on both LC_STATUS and SITE
dfDT["LC_STATUS_SITE"] = dfDT["LC_STATUS"] + "_" + dfDT["SITE"].astype(str)

df_trainDT, df_testDT, df_holdoutDT = straify_train_test_holdout(dfDT, 'LC_STATUS', 'LC_STATUS_SITE', test_size = 0.2, holdout_size = 0.2, random_state = 42)

Verfiy stratification accurate

In [ ]:
pd.crosstab(df_trainDT["LC_STATUS"], df_trainDT["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.134752,0.000000,0.009756,0.0125,0.029610
LC_NEG,0.730496,0.685355,0.209756,0.0125,0.753701
LC_POS,0.134752,0.314645,0.780488,0.9750,0.216689


In [30]:
pd.crosstab(df_testDT["LC_STATUS"], df_testDT["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.176471,0.000000,0.00000,0.014925,0.875
LC_NEG,0.470588,0.683478,0.19697,0.014925,0.125
LC_POS,0.352941,0.316522,0.80303,0.970149,0.000


In [31]:
pd.crosstab(df_holdoutDT["LC_STATUS"], df_holdoutDT["SITE"], normalize="columns")

SITE,1,2,3,4,5
LC_STATUS,,,,,
HC,0.176471,0.000000,0.00000,0.015038,0.8
LC_NEG,0.470588,0.682927,0.19697,0.007519,0.1
LC_POS,0.352941,0.317073,0.80303,0.977444,0.1


## Export

In [33]:
datasets.arrow_dataset.Dataset.from_pandas(df_train81,preserve_index=False).save_to_disk(
    "../Data/DataV2/TTS/T81_Train.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(df_test81,preserve_index=False).save_to_disk(
    "../Data/DataV2/TTS/T81_Test.arrow"
)
datasets.arrow_dataset.Dataset.from_pandas(df_holdout81,preserve_index=False).save_to_disk(
    "../Data/DataV2/TTS/T81_Holdout.arrow"
)

Saving the dataset (0/1 shards):   0%|          | 0/2814 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/938 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/939 [00:00<?, ? examples/s]

In [34]:
datasets.arrow_dataset.Dataset.from_pandas(
    df_train85, preserve_index=False
).save_to_disk("../Data/DataV2/TTS/T85_Train.arrow")
datasets.arrow_dataset.Dataset.from_pandas(
    df_test85, preserve_index=False
).save_to_disk("../Data/DataV2/TTS/T85_Test.arrow")
datasets.arrow_dataset.Dataset.from_pandas(
    df_holdout85, preserve_index=False
).save_to_disk("../Data/DataV2/TTS/T85_Holdout.arrow")

Saving the dataset (0/1 shards):   0%|          | 0/3237 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1079 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1080 [00:00<?, ? examples/s]

In [35]:
datasets.arrow_dataset.Dataset.from_pandas(
    df_trainDT, preserve_index=False
).save_to_disk("../Data/DataV2/TTS/DT_Train.arrow")
datasets.arrow_dataset.Dataset.from_pandas(
    df_testDT, preserve_index=False
).save_to_disk("../Data/DataV2/TTS/DT_Test.arrow")
datasets.arrow_dataset.Dataset.from_pandas(
    df_holdoutDT, preserve_index=False
).save_to_disk("../Data/DataV2/TTS/DT_Holdout.arrow")

Saving the dataset (0/1 shards):   0%|          | 0/2400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/800 [00:00<?, ? examples/s]